In [ ]:
# Kaggle Standard Setup Cell
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image.

import numpy as np
import pandas as pd
import os

# List available files in /kaggle/input
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
train=pd.read_csv('/Users/piyushagarwal/Documents/Python Files/playground-series-s4e2/train.csv')
test=pd.read_csv('/Users/piyushagarwal/Documents/Python Files/playground-series-s4e2/test.csv')
sol=pd.read_csv('/Users/piyushagarwal/Documents/Python Files/playground-series-s4e2/solution.csv')

In [ ]:
train.info()

In [ ]:
train.drop_duplicates()

In [ ]:
train.nunique()

In [ ]:
train['NObeyesdad'].value_counts()

In [ ]:
num_col=train.select_dtypes(include=['int64','float64'])
cat_col=train.select_dtypes(include=['object']).columns

In [ ]:
num_col.columns.tolist()

In [ ]:
num_col.info()

In [ ]:
cat_col.tolist()

In [ ]:
for col in num_col:
    plt.figure(figsize=(12,6))
    sns.histplot(x=col,data=train,kde=True)
    plt.xlabel(col)
    plt.ylabel('NObeyesdad')
    plt.show()

In [ ]:
for col in cat_col:
    plt.figure(figsize=(12,6))
    sns.countplot(x=col,data=train)
    plt.xlabel(col)
    plt.ylabel('NObeyesdad')
    plt.show()

In [ ]:
for col in num_col:
    plt.figure(figsize=(12,6))
    sns.boxplot(x=col,data=train)
    plt.xlabel(col)
    plt.ylabel('NObeyesdad')
    plt.show()

In [ ]:
col = "Age"

Q1 = train[col].quantile(0.25)
Q3 = train[col].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df_clean = train[(train[col] >= lower) & (train[col] <= upper)]

In [ ]:
# for col in num_col:
#     plt.figure(figsize=(12,6))
#     sns.boxplot(x=col,data=df_clean)
#     plt.xlabel(col)
#     plt.ylabel('NObeyesdad')
#     plt.show()

In [ ]:
# train=df_clean
# train.info()

In [ ]:
train.drop(['NCP'],axis=1,inplace=True)

In [ ]:
num_col=train.select_dtypes(include=['int64','float64']).columns
cat_col=train.select_dtypes(include=['object']).columns

In [ ]:
for col in cat_col:
    print(train[col].value_counts())

In [ ]:
# def cap_outliers_categorywise_all(df, cat_col, num_cols):

#     def cap_group(group):
#         group = group.copy()

#         # Work only on numeric columns
#         for col in num_cols:
#             Q1 = group[col].quantile(0.25)
#             Q3 = group[col].quantile(0.75)
#             IQR = Q3 - Q1

#             lower = Q1 - 1.5 * IQR
#             upper = Q3 + 1.5 * IQR

#             group[col] = group[col].clip(lower, upper)

#         return group

#     return (
#         df.groupby(cat_col, group_keys=False, observed=True, sort=False)
#           .apply(lambda g: cap_group(g))
#     )

In [ ]:
# df_capped = cap_outliers_categorywise_all(train, 'NObeyesdad', num_col)

In [ ]:
# target='NObeyesdad'
# for col in num_col:
#     plt.figure(figsize=(12,6))
#     sns.boxplot(x=target,y=col,data=df_capped)
#     plt.xlabel(target)
#     plt.ylabel(col)
#     plt.show()

In [ ]:
# train=df_capped

In [ ]:
for col in num_col:
    median_val = train[col].median()
    train.fillna({col: median_val}, inplace=True)
    print(f"Filled missing in {col} with median: {median_val}")

for col in cat_col:
    mode_val = train[col].mode()[0]
    train.fillna({col: mode_val}, inplace=True)
    print(f"Filled missing in {col} with mode: {mode_val}")

In [ ]:
from sklearn.preprocessing import RobustScaler
binary_cols = [
    'family_history_with_overweight',
    'FAVC',
    'SMOKE',
    'SCC'
]
multiclass_cols = ['CAEC', 'CALC', 'MTRANS']
target_col = 'NObeyesdad'

In [ ]:
for col in binary_cols:
    train[col] = train[col].map({'no': 0, 'yes': 1})

In [ ]:
train['Gender'] = train['Gender'].map({'Male': 1, 'Female': 0})

In [ ]:
train = pd.get_dummies(train, columns=multiclass_cols, drop_first=True,dtype=int)

In [ ]:
correct_order = [
    'Insufficient_Weight',
    'Normal_Weight',
    'Overweight_Level_I',
    'Overweight_Level_II',
    'Obesity_Type_I',
    'Obesity_Type_II',
    'Obesity_Type_III'
]
le = LabelEncoder()
le.classes_ = np.array(correct_order)
train[target_col] = le.transform(train[target_col])

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train[num_col].dtypes

In [ ]:
scaler = RobustScaler()
train[num_col] = scaler.fit_transform(train[num_col])

In [ ]:
corr=train.corr(numeric_only=True)
plt.figure(figsize=(14,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap", fontsize=14)
plt.show()

In [ ]:
X = train.drop('NObeyesdad', axis=1)
y = train['NObeyesdad']

In [ ]:
train['NObeyesdad'].value_counts()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from sklearn.metrics import accuracy_score ,f1_score, confusion_matrix

In [ ]:
# import optuna
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score, StratifiedKFold

# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
#         'max_depth': trial.suggest_int('max_depth', 3, 30),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
#         'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
#         'oob_score': trial.suggest_categorical('oob_score', [True, False]),
#         'n_jobs': -1
#     }

#     model = RandomForestClassifier(**params)

#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    
#     return scores.mean()

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)  # increase n_trials for better search

# print("Best Accuracy: {:.4f}".format(study.best_value))
# print("Best Parameters: ", study.best_params) 

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier()

# param_grid = {
#      'n_estimators': [100, 200, 500],           
#      'max_depth': [None, 10, 20, 30],           
#      'min_samples_split': [2, 5, 10],           
#      'min_samples_leaf': [1, 2, 4],             
#      'max_features': ['sqrt', 'log2', None],
#      'oob_score':[True],
#      'class_weight':['balanced']
# }

# grid = GridSearchCV(
#       estimator=model,
#       param_grid=param_grid,
#       scoring='accuracy',    
#       cv=3,                  
#       verbose=2,
#       n_jobs=-1
# )              


# grid.fit(X_train, y_train)
# print("Best Parameters:", grid.best_params_)
# print("Best Cross-Validation Score:", grid.best_score_)
# best_model = grid.best_estimator_
# y_pred = best_model.predict(X_val)

# The best parameters from above grid search are:
# {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=426, max_depth=22, min_samples_split=6, min_samples_leaf=1, max_features=None, oob_score=False)
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred_train)
print(f"Training Accuracy: {train_accuracy:.4f}")   
train_f1score = f1_score(y_train, y_pred_train,average='weighted')
print(f"Training F1 Score: {train_f1score:.4f}")
train_conf_matrix = confusion_matrix(y_train, y_pred_train)
print("Training Confusion Matrix:")
print(train_conf_matrix)
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
f1score = f1_score(y_val, y_pred,average='weighted')
conf_matrix = confusion_matrix(y_val, y_pred)
print(f"Val Accuracy: {accuracy:.4f}")
print(f"VAl F1 Score: {f1score:.4f}")
print("Val Confusion Matrix:")
print(conf_matrix)

In [ ]:
rf=RandomForestClassifier(n_estimators=426, max_depth=22, min_samples_split=6, min_samples_leaf=1, max_features=None, oob_score=False)
rf.fit(X, y)

In [ ]:
test.info()

In [ ]:
test.drop_duplicates()

In [ ]:
test.drop(columns=['id'],inplace=True)

In [ ]:
# test["Age"] = test["Age"].clip(upper=35)

In [ ]:
test.drop(['NCP'],axis=1,inplace=True)

In [ ]:
num_col_test=test.select_dtypes(include=['int64','float64']).columns
cat_col_test=test.select_dtypes(include=['object']).columns

In [ ]:
binary_cols = [
    'family_history_with_overweight',
    'FAVC',
    'SMOKE',
    'SCC'
]
multiclass_cols = ['CAEC', 'CALC', 'MTRANS']

In [ ]:
for col in binary_cols:
    test[col] = test[col].map({'no': 0, 'yes': 1})

In [ ]:
test['Gender'] = test['Gender'].map({'Male': 1, 'Female': 0})

In [ ]:
test = pd.get_dummies(test, columns=multiclass_cols, drop_first=True,dtype=int)

In [ ]:
test.info()

In [ ]:
test[num_col_test] = scaler.transform(test[num_col_test])
test.head()

In [ ]:
test_predictions = rf.predict(test)

In [ ]:
sol.drop(['id','Usage'], axis=1, inplace=True)

In [ ]:
sol['NObeyesdad'].value_counts()

In [ ]:
sol['NObeyesdad']=le.transform(sol['NObeyesdad'])
sol.head()

In [ ]:
accuracy=accuracy_score(sol['NObeyesdad'], test_predictions)
print(f"Test Set Accuracy: {accuracy:.4f}")
f1score_test = f1_score(sol['NObeyesdad'], test_predictions, average='weighted')
print(f"Test Set F1 Score: {f1score_test:.4f}")
test_conf_matrix = confusion_matrix(sol['NObeyesdad'], test_predictions)
print("Test Set Confusion Matrix:")
print(test_conf_matrix)

In [ ]:
test_predictions

In [ ]:
test_predictions_labels = le.inverse_transform(test_predictions)
test_predictions_labels

In [ ]:
sandbox=pd.DataFrame(test_predictions_labels, columns=['NObeyesdad'])

In [ ]:
sandbox.head()

In [ ]:
sandbox.to_csv('sandbox.csv', index=False)